# Correlation power analysis on pyecsca emulated traces

### Setup

In [ ]:
from pyecsca.ec.mult import LTRMultiplier
from pyecsca.ec.mod import Mod
from pyecsca.ec.point import Point, InfinityPoint
from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.curve import EllipticCurve
from pyecsca.ec.params import DomainParameters
from pyecsca.sca.attack.leakage_model import HammingWeight
from pyecsca.sca.target.emulator import EmulatorTarget
from random import randint
from pyecsca.sca.attack.CPA import CPA

In [ ]:
model = ShortWeierstrassModel()
coords = model.coordinates["projective"]
p = 0xd7d1247f
a = Mod(0xa4a44016, p)
b = Mod(0x73f76716, p)
n = 0xd7d2a475
h = 1
gx, gy, gz = Mod(0x54eed6d7, p), Mod(0x6f1e55ac, p), Mod(1, p)
generator = Point(coords, X=gx, Y=gy, Z=gz)
neutral = InfinityPoint(coords)

curve = EllipticCurve(model, coords, p, neutral, {"a": a, "b": b})
params = DomainParameters(curve, generator, n, h)

add = coords.formulas["add-2015-rcb"]
dbl = coords.formulas["dbl-2015-rcb"]
scl = coords.formulas["z"]

mult = LTRMultiplier(add, dbl, None)

In [ ]:
scalar_bit_length = 8
secret_scalar = randint(1 << (scalar_bit_length - 1), (1 << scalar_bit_length) - 1)
print(secret_scalar)

In [ ]:
mult.init(params, generator)
real_pub_key = mult.multiply(secret_scalar)

### Emulation of traces

In [ ]:
num_of_traces = 500

In [ ]:
emulator = EmulatorTarget(model, coords, mult)
emulator.set_params(params)
emulator.set_leakage_model(HammingWeight())

In [ ]:
points, traces = emulator.emulate_scalar_mult_traces(num_of_traces, secret_scalar)

### CPA

In [ ]:
cpa = CPA(points, traces, HammingWeight(), mult, params)

#### Recover the 8-bit scalar bit by bit

In [ ]:
recovered_scalar = 128 #1000 0000

In [ ]:
recovered_scalar = cpa.recover_bit(recovered_scalar, 1, scalar_bit_length, real_pub_key)
print(f"Recovered scalar after recovering 2. bit of the secret scalar: {recovered_scalar} = {recovered_scalar:b}")

In [ ]:
# We can do this for each bit of the scalar
for i in range(2, scalar_bit_length):
    recovered_scalar = cpa.recover_bit(recovered_scalar, i, scalar_bit_length, real_pub_key)
    print(f"Recovered scalar after recovering {i + 1}. bit of the secret scalar: {recovered_scalar} = {recovered_scalar:b}")
print(recovered_scalar == secret_scalar)

#### Recover the whole 8-bit scalar

In [ ]:
recovered_scalar = cpa.perform(scalar_bit_length, real_pub_key)
print(recovered_scalar)
print(recovered_scalar == secret_scalar)